Enforce Schema but no data validation

In [1]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")
llm=ChatGroq(model="gemma2-9b-it")

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser,ResponseSchema

In [5]:
schema=[
    ResponseSchema(name="name",description="Name of the character"),
    ResponseSchema(name="age",description="Age of the character"),
    ResponseSchema(name="city",description="Age of the character"),
    ResponseSchema(name="role",description="Role of the character"),
    ResponseSchema(name="description",description="Description of the character")
]

In [6]:
parser=StructuredOutputParser.from_response_schemas(schema)
temp=PromptTemplate(
    template="Give details of the character: {character}\n {format_instruction}",
    input_variables=["character"],
    partial_variables={"format_instruction":parser.get_format_instructions()}
)
temp

PromptTemplate(input_variables=['character'], input_types={}, partial_variables={'format_instruction': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"name": string  // Name of the character\n\t"age": string  // Age of the character\n\t"city": string  // Age of the character\n\t"role": string  // Role of the character\n\t"description": string  // Description of the character\n}\n```'}, template='Give details of the character: {character}\n {format_instruction}')

In [7]:
parser

StructuredOutputParser(response_schemas=[ResponseSchema(name='name', description='Name of the character', type='string'), ResponseSchema(name='age', description='Age of the character', type='string'), ResponseSchema(name='city', description='Age of the character', type='string'), ResponseSchema(name='role', description='Role of the character', type='string'), ResponseSchema(name='description', description='Description of the character', type='string')])

In [10]:
temp.invoke({"character":"Bruce Wayne"})

StringPromptValue(text='Give details of the character: Bruce Wayne\n The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"name": string  // Name of the character\n\t"age": string  // Age of the character\n\t"city": string  // Age of the character\n\t"role": string  // Role of the character\n\t"description": string  // Description of the character\n}\n```')

In [9]:
chain= temp | llm | parser

In [11]:
chain.invoke({"character" : "Bruce Wayne"})

{'name': 'Bruce Wayne',
 'age': 'Mid-40s',
 'city': 'Gotham City',
 'role': 'Vigilante, Philanthropist',
 'description': 'A wealthy industrialist and the masked vigilante Batman, Bruce Wayne is driven by the tragic murder of his parents as a child. Haunted by their loss, he dedicates his life to fighting crime and protecting Gotham City from corruption and injustice.'}